In [10]:
import sys
sys.path.append('..')
import time
from astropy.table import Table
import pyfits as fits
import numpy as np
import matplotlib.pyplot as plt
from os.path import expanduser
home = expanduser("~")

In [11]:
download_path=home+'/Desktop/' # To be adjusted on your machine

export_path=home +'/Desktop/' # To be adjusted on your machine

In [12]:
user_imgs = raw_input("Please input # of images: ")
user_epochs = raw_input("Please input # of epochs: ")

imgs = int(user_imgs)
n_epochs = int(user_epochs)

Please input # of images: 21
Please input # of epochs: 4


In [13]:
cat = Table.read(download_path+'GroundBasedTraining/classifications.csv')
cat= cat[0:imgs]

In [15]:
ims = np.zeros((imgs, 3, 101, 101))
counter = 0


for i, id in enumerate(cat['ID']):
    if (i%10 == 0) and (i!=0):
        counter +=1
        print str(counter)+"0 done."
    for j, b in enumerate(['R', 'I', 'G']):
        ims[i, j] = fits.getdata(download_path+'GroundBasedTraining/Public/Band'+str(j+1)+'/imageSDSS_'+b+'-'+str(id)+'.fits')
        cat['image'] = ims

cat.write(export_path+'catalogs_'+str(imgs)+'_RGB.hdf5', path='/ground', append=True)
print "Done !"

10 done.
20 done.


IOError: Table /ground already exists

In [6]:
from astropy.table import Table

d = Table.read(export_path+'catalogs_'+str(imgs)+'_RGB.hdf5', path='/ground')  # Path to be adjusted on your machine
print d['image'].shape

x = np.asarray(d['image']).reshape((-1,3,101,101))
print x.shape

y = np.asarray(d['is_lens']).reshape((-1,1))
print y.shape

xval = np.asarray(d['image'][int(imgs*.75):]).reshape((-1,3,101,101))
yval = np.asarray(d['is_lens'][int(imgs*.75):]).reshape((-1,1))


IOError: [Errno 2] No such file or directory: '/Users/Chris/Desktop/catalogs_20_RGB.hdf5'

In [ ]:
vmin=-1e-9
vmax=1e-9
scale=100

mask = np.where(x == 100)
mask_val = np.where(xval == 100)

x[mask] = 0
xval[mask_val] = 0


x = np.clip(x, vmin, vmax)/vmax * scale
xval = np.clip(xval, vmin, vmax)/vmax * scale 

x[mask] = 0
xval[mask_val] = 0

In [ ]:
print 'Lens(1): '+ str(np.sum(y==1))
print 'Non-lens(0): '+ str(np.sum(y==0))

In [ ]:
import datetime

print(datetime.datetime.now())

from deeplens.resnet_classifier import deeplens_classifier
print 'Training begining...'
print str(n_epochs)+' epochs...'
print str(imgs) + ' images...'

model = deeplens_classifier(learning_rate=0.001, learning_rate_steps=3, learning_rate_drop=0.1, batch_size=128, n_epochs=n_epochs)           

model.fit(x,y,xval,yval) 

model.save('deeplens_params_final.npy')

print(datetime.datetime.now())
print 'Completed Training!'